# Importação das Bibliotecas

In [31]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy import stats
from IPython.display import display
from PIL import Image

# Leitura e Análise da Base

In [3]:
# Leitura da planilha 
DADOS = r"C:\Users\rodri\OneDrive\Sincronização\Documentos\GitHub\confidence_interval_marketplace_sales\data\olist_order_items_dataset.xlsx"
df = pd.read_excel(DADOS)

# Visualizar as primeiras linhas da tabela
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,9/19/17 9:45,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-03-05 11:05:00,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,1/18/18 14:48,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,8/15/18 10:10,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2/13/17 13:57,199.90,18.14


In [4]:
# Visualizar as últimas linhas da tabela
df.tail()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-02-05 04:11:00,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,7/20/18 4:31,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,10/30/17 17:14,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,8/21/17 0:04,55.99,8.72
112649,fffe41c64501cc87c801fd61db3f6244,1,350688d9dc1e75ff97be326363655e01,f7ccf836d21b2fb1de37564105216cc1,2018-12-06 17:10:00,43.00,12.79


In [5]:
# Conferir os tipos de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


# Problema de Negócio

Uma empresa presente nos maiores marketplaces do Brasil resolveu fazer um estudo para avaliar o valor dos produtos vendidos e o valor do frete. Para isso, coletou uma amostra de mais de 100 mil vendas com seus respectivos fretes.

O estudo teve como objetivo:
- calcular a estimativa pontual do preço médio dos pedidos e do valor médio do frete
- calcular os intervalos de confiança para preço e frete considerando o coeficiente de confiança igual a 95%
- definir a probabilidade de que esses intervalos de confiança contenham o verdadeiro valor da média populacional

## Estatísticas Básicas

In [6]:
# Definição da variáveis das amostras
amostra_price = df["price"].to_numpy()
amostra_freight = df["freight_value"].to_numpy()

# Nível de confiança de 95%
conf_95 = 0.95

In [39]:
import os
print(os.getcwd())  # Mostra o diretório onde o notebook está rodando
print(os.listdir("../images"))  # Lista os arquivos dentro da pasta "images"

C:\Users\rodri\OneDrive\Sincronização\Documentos\GitHub\confidence_interval_marketplace_sales\notebooks
[]


In [40]:
import os
print(os.path.exists("../images"))  # Verifica se a pasta existe

True


In [38]:
image = Image.open("../images/intervalo_confianca.jpg")  # Volta um diretório para acessar "images"
display(image)

FileNotFoundError: [Errno 2] No such file or directory: '../images/intervalo_confianca.jpg'

In [34]:
import os
print(os.listdir("images"))

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'images'

# Resolução

In [18]:
# Estatísticas básicas da variável 'preço'
n_price = len(amostra_price)
media_price = amostra_price.mean()
variancia_amostral_price = np.var(amostra_price, ddof=1)
t_critico_price = abs(stats.t.ppf((1 - conf_95) / 2, df=n_price-1))
erro_padrao_price = amostra_price.std(ddof=1) / np.sqrt(n_price)

# Exibir os resultados
print(f"Tamanho da Amostra: {n_price}")
print(f"Preço Médio: {media_price:.2f}")
print(f"Variância Amostral: {variancia_amostral_price:.2f}")
print(f"Erro Padrão: {erro_padrao_price:.2f}")
print(f"T-1: {t_critico_price:.2f}")

Tamanho da Amostra: 112650
Preço Médio: 120.65
Variância Amostral: 33721.42
Erro Padrão: 0.55
T-1: 1.96


In [28]:
# Função para calcular intervalo de confiança do preço
def intervalo_confianca(conf, media, erro_padrao, n):
    t_critico = abs(stats.t.ppf((1 - conf) / 2, df=n-1))
    limite_inferior = media - t_critico * erro_padrao
    limite_superior = media + t_critico * erro_padrao
    return t_critico, limite_inferior, limite_superior

# Intervalo de Confiança
t_critico, limite_inferior, limite_superior = intervalo_confianca(conf_95, media_price, erro_padrao_price, n_price)

# Exibir resultados
print("\nIntervalo de Confiança de 95%:")
print(f"- T-Crítico: {t_critico:.2f}")
print(f"- Limite Inferior: {limite_inferior:.2f}")
print(f"- Limite Superior: {limite_superior:.2f}")


Intervalo de Confiança de 95%:
- T-Crítico: 1.96
- Limite Inferior: 119.58
- Limite Superior: 121.73


In [29]:
# Estatísticas básicas da variável 'preço'
n_freight = len(amostra_freight)
media_freight = amostra_freight.mean()
variancia_amostral_freight = np.var(amostra_freight, ddof=1)
t_critico_freight = abs(stats.t.ppf((1 - conf_95) / 2, df=n_freight-1))
erro_padrao_freight = amostra_freight.std(ddof=1) / np.sqrt(n_freight)

# Exibir os resultados
print(f"Tamanho da Amostra: {n_freight}")
print(f"Preço Médio: {media_freight:.2f}")
print(f"Variância Amostral: {variancia_amostral_freight:.2f}")
print(f"Erro Padrão: {erro_padrao_freight:.2f}")
print(f"T-1: {t_critico_freight:.2f}")

Tamanho da Amostra: 112650
Preço Médio: 19.99
Variância Amostral: 249.84
Erro Padrão: 0.05
T-1: 1.96
